In [ ]:
#pip install yelpapi
#pip install ipyleaflet
#!pip install dash
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2023-04-24 05:22:06--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  42.2MB/s    in 0.3s    

2023-04-24 05:22:06 (42.2 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
!cat /path/to/logfile.log

!./ngrok http 8000


cat: /path/to/logfile.log: No such file or directory
^C


In [ ]:
from yelpapi import YelpAPI
import folium

api_key = 'gBfqGZjS05a_0Mzi908uHV7n3T_V1DHNI6ifx9YAexKoL4FI7DX8ktx-dXO9n6UF3Ly64RXgG-bbINKWTptxHAynH5D5BkYSOt5SVtm-9RsKfqWGAlC_7XZURP1FZHYx'

yelp_api = YelpAPI(api_key)

locations = ['Manhattan', 'Queens', 'Brooklyn']
category = 'restaurants'

ny_map = folium.Map(location=[40.7128, -74.0060], zoom_start=10)

for location in locations:
    response = yelp_api.search_query(location=location, categories=category, limit=50)

    for business in response['businesses']:
        name = business['name']
        lat = business['coordinates']['latitude']
        lng = business['coordinates']['longitude']
        popup_text = f"{name}"
        popup = folium.Popup(popup_text, max_width=300, min_width=200)
        folium.Marker([lat, lng], popup=popup).add_to(ny_map)

ny_map

In [ ]:
from flask import Flask, render_template, request
from yelpapi import YelpAPI
import folium

app = Flask(__name__)

# Define tus credenciales de Yelp API
api_key = 'gBfqGZjS05a_0Mzi908uHV7n3T_V1DHNI6ifx9YAexKoL4FI7DX8ktx-dXO9n6UF3Ly64RXgG-bbINKWTptxHAynH5D5BkYSOt5SVtm-9RsKfqWGAlC_7XZURP1FZHYx'

# Inicializa la instancia de Yelp API
yelp_api = YelpAPI(api_key)

# Define las categorías de negocios que deseas buscar
categories = {
    'Restaurants': 'restaurants',
    'Tiendas y comercios': 'shopping',
    'Salud y medicina': 'health',
    'Bares y vida nocturna': 'nightlife',
    'Educación': 'education',
    'Comida para llevar y entrega a domicilio': 'fooddeliveryservices'
}

# Define los distritos que deseas buscar
locations = {
    'Manhattan': [40.7831, -73.9712],
    'Queens': [40.7282, -73.7949],
    'Brooklyn': [40.6782, -73.9442]
}

# Define el mapa inicial centrado en Nueva York
initial_map = folium.Map(location=[40.7128, -74.0060], zoom_start=10)

# Define la ruta principal
@app.route('/')
def index():
    # Renderiza la plantilla index.html y pasa los filtros y el mapa inicial
    return render_template('index.html', categories=categories, locations=locations, map=initial_map._repr_html_())

# Define la ruta para procesar el formulario y mostrar los resultados en el mapa
@app.route('/', methods=['POST'])
def search():
    # Obtiene los filtros seleccionados del formulario
    selected_category = request.form.get('category')
    selected_location = request.form.get('location')

    # Realiza la búsqueda de negocios para la ubicación y categoría seleccionadas
    response = yelp_api.search_query(location=selected_location, categories=categories[selected_category], limit=50)

    # Crea un mapa centrado en la ubicación seleccionada
    search_map = folium.Map(location=locations[selected_location], zoom_start=12)

    # Agrega un marcador en el mapa para cada negocio encontrado
    for business in response['businesses']:
        name = business['name']
        lat = business['coordinates']['latitude']
        lng = business['coordinates']['longitude']
        popup_text = f"{name}"
        popup = folium.Popup(popup_text, max_width=300, min_width=200)
        folium.Marker([lat, lng], popup=popup).add_to(search_map)

    # Renderiza la plantilla index.html y pasa los filtros y el mapa de búsqueda
    return render_template('index.html', categories=categories, locations=locations, map=search_map._repr_html_())

if __name__ == '__main__':
    app.run(port=8000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8000
INFO:werkzeug:Press CTRL+C to quit
